In [298]:
from keras.layers import *
from keras import Model
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras import Sequential
import keras as kr
import keras.losses as losses
 
 
# Setup the network parameters:
intermediate_dim = 300
latent_dim = 15
beta = 5
epochs = 15
n_sequences = 101
n_features = 7

# 
# def transform_to_lists(data):
#     list_v = []
#     for i in data:
#         list_v.append(list(i))
#     return list_v
#     
# def read_nominal_sequences():
#     sequences = []
#     scaler = StandardScaler()
#     for i in range(n_sequences):
#         name = "Data/Boat_nominal_data/sequences/Boat_sequence^"+str(i)+".csv"
#         data = pd.read_csv(name)
#         data = data.drop(columns=["Unnamed: 0"])
#         data = transform_to_lists(scaler.fit_transform(data))
#         sequences.append(data)
#     return sequences
# 
# 
# nominal_sequences = read_nominal_sequences()
# nominal_sequences = np.array(nominal_sequences)
# print(type(nominal_sequences), type(nominal_sequences[0]), type(nominal_sequences[0][0]))

boat_csv = pd.read_csv("Data/Boat_nominal_data/Boat_sequences.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0"])
# boat_csv = boat_csv[:5739]
scaler = StandardScaler()
normal_data = scaler.fit_transform(boat_csv)
print(normal_data.shape)

boat_curved = pd.read_csv("Data/Anomalous_boat_data.csv")
boat_curved= boat_curved.drop(columns=["Unnamed: 0", "heading"])    
scaler = StandardScaler()
anomalous_data = scaler.fit_transform(boat_curved)
print(anomalous_data.shape)


(66963, 7)
(6620, 7)


In [299]:
batch_size =  663
interval = 663
def prepare_sequences(data):
    samples = []
    for i in range(0,data.shape[0]- batch_size, interval):
        sample = data[i:i+batch_size]	
        samples.append(sample)

    sequences = np.array(samples)

    # Batch size (Number of samples time steps and number of features
    trainX = np.reshape(sequences, (len(sequences), batch_size, 7))

    return trainX

trainX_nominal = prepare_sequences(normal_data) 
print(trainX_nominal.shape)

input_length = trainX_nominal.shape[0]
trainX_anomalous = prepare_sequences(anomalous_data)
print(trainX_anomalous.shape)  


(100, 663, 7)
(9, 663, 7)


In [300]:
model = Sequential()

encoder = Sequential()
encoder.add(Conv1D(filters=100, input_shape=(None,7), padding='same', 
                   kernel_size=n_features))
encoder.add(Conv1D(filters=200, padding='same', 
                   kernel_size=n_features))
encoder.add(Conv1D(filters=100,padding='same', 
                   kernel_size=n_features))
encoder.add(Conv1D(filters=50, padding='same',
                   kernel_size=n_features))
encoder.add(Conv1D(filters=latent_dim, padding='same',
                   kernel_size=n_features))

decoder = Sequential()
decoder.add(Dense(64))
decoder.add(Dense(units=n_features))

model.add(encoder)
model.add(decoder)

model.compile(optimizer="adam", loss="mse", metrics=['accuracy'])

model.fit(x=trainX_nominal, y=trainX_nominal, epochs=epochs)


Epoch 1/15


 32/100 [========>.....................] - ETA: 7s - loss: 1.2115 - acc: 0.0819

 64/100 [==================>...........] - ETA: 2s - loss: 1.0573 - acc: 0.2260

 96/100 [===========================>..] - ETA: 0s - loss: 0.8842 - acc: 0.3027

100/100 [==============================] - 5s 51ms/step - loss: 0.8632 - acc: 0.3107


...

Epoch 15/15


 32/100 [========>.....................] - ETA: 2s - loss: 0.0051 - acc: 0.9592

 64/100 [==================>...........] - ETA: 1s - loss: 0.0051 - acc: 0.9587

 96/100 [===========================>..] - ETA: 0s - loss: 0.0051 - acc: 0.9586

100/100 [==============================] - 3s 33ms/step - loss: 0.0051 - acc: 0.9588


In [301]:
from sklearn.manifold import TSNE

def tsne(enc_input):
    tsne = TSNE(n_components=2, random_state=0)
    
    tsne_obj= tsne.fit_transform(enc_input)
    tsne_df = pd.DataFrame({'X':tsne_obj[:,0],
                            'Y':tsne_obj[:,1],
                            })
    
    plt.scatter(x=tsne_df["X"],y=tsne_df["Y"], alpha=0.5)
    plt.show()
    
    return tsne_df


In [166]:

def check_disentanglement(enc, dec, autoenc, weight_modifiers):
    nom_enc =(enc.predict(trainX_nominal))
    print(nom_enc.shape)     
    # print(len(nom_enc))
    
    tsne(np.average(nom_enc, axis=0))
    # 
    # nom_enc_modified = nom_enc
    # for i, elem in enumerate(nom_enc_modified):
    #     nom_enc_modified[i] = elem * weight_modifiers    
    # 
    nom_dec = dec.predict(nom_enc)
    tsne(np.average(nom_dec, axis=0))
          
    # df_nominal = pd.DataFrame(normal_data, columns=boat_csv.columns)
    # 
    # df = pd.DataFrame(nom_dec[0], columns=boat_csv.columns)
    # 
    # run_lenght = 663
    # 
    # plt.subplot(421)
    # plt.plot(df["Lon"][:run_lenght])
    # plt.plot(df_nominal["Lon"][:run_lenght], 'r')
    # plt.title("Longitude")
    # 
    # plt.subplot(422)
    # plt.plot(df["Lat"][:run_lenght])
    # plt.plot(df_nominal["Lat"][:run_lenght], 'r')
    # plt.title("latitude")
    # 
    # plt.subplot(423)
    # plt.plot(df["Speed"][:run_lenght])
    # plt.plot(df_nominal["Speed"][:run_lenght], 'r')
    # plt.title("Speed")        
    # 
    # plt.subplot(424)
    # plt.plot(df["Acceleration"][:run_lenght])
    # plt.plot(df_nominal["Acceleration"][:run_lenght], 'r')
    # plt.title("Acceleration")         
    # 
    # plt.subplot(425)
    # plt.plot(df["Degrees"][:run_lenght])
    # plt.plot(df_nominal["Degrees"][:run_lenght], 'r')
    # plt.title("Degrees")           
    # 
    # plt.subplot(426)
    # plt.plot(df["M0C"][:run_lenght])
    # plt.plot(df_nominal["M0C"][:run_lenght], 'r')
    # plt.title("M0C")
    #         
    # plt.subplot(427)
    # plt.plot(df["M1C"][:run_lenght])
    # plt.plot(df_nominal["M1C"][:run_lenght], 'r')
    # plt.title("M1C")
    #  
    # plt.show()
    # 
    # plt.subplot(421)
    # plt.plot(df["Lon"][:5739] - df_nominal["Lon"][:run_lenght])
    # plt.title("Longitude_difference")
    # 
    # plt.subplot(422)
    # plt.plot(df["Lat"][:5739] - df_nominal["Lat"][:run_lenght])
    # plt.title("latitude_difference")
    # 
    # plt.subplot(423)
    # plt.plot(df["Speed"][:5739] - df_nominal["Speed"][:run_lenght])
    # plt.title("Speed_difference")        
    # 
    # plt.subplot(424)
    # plt.plot(df["Acceleration"][:run_lenght] - df_nominal["Acceleration"][:run_lenght])
    # plt.title("Acceleration_difference")         
    # 
    # plt.subplot(425)
    # plt.plot(df["Degrees"][:run_lenght] - df_nominal["Degrees"][:run_lenght])
    # plt.title("Degrees_difference")           
    # 
    # plt.subplot(426)
    # plt.plot(df["M0C"][:run_lenght] - df_nominal["M0C"][:run_lenght])
    # plt.title("M0C_difference")
    #         
    # plt.subplot(427)
    # plt.plot(df["M1C"][:run_lenght] - df_nominal["M1C"][:run_lenght])
    # plt.title("M1C_difference")
    # 
    # plt.show()


Epoch 1/15


 32/100 [========>.....................] - ETA: 5s - loss: 1.1587 - acc: 0.0969

 64/100 [==================>...........] - ETA: 1s - loss: 1.1188 - acc: 0.2623

 96/100 [===========================>..] - ETA: 0s - loss: 0.8959 - acc: 0.3419

100/100 [==============================] - 4s 43ms/step - loss: 0.8816 - acc: 0.3448


...

Epoch 15/15


 32/100 [========>.....................] - ETA: 1s - loss: 0.0052 - acc: 0.9527

 64/100 [==================>...........] - ETA: 0s - loss: 0.0051 - acc: 0.9533

 96/100 [===========================>..] - ETA: 0s - loss: 0.0051 - acc: 0.9542

100/100 [==============================] - 3s 27ms/step - loss: 0.0051 - acc: 0.9542


In [322]:
weight_modifiers = np.ones((latent_dim,))

nom_enc =(encoder.predict(trainX_nominal))

# weight_modifiers[0] = 10
# nom_enc_modified = nom_enc
# for i, elem in enumerate(nom_enc_modified):
#     nom_enc_modified[i] = elem * weight_modifiers    
tsne_enc_df = tsne(np.average(nom_enc, axis=0))


nom_dec = decoder.predict(nom_enc)
tsne_dec_df = tsne(np.average(nom_dec, axis=0))

In [318]:
from sklearn.cluster import KMeans
cluster_comp = [0,1,2,3]# print(tsne_enc_df)
# print(tsne_dec_df)

kmeans_normal = KMeans(n_clusters=4, random_state=0).fit(np.average(trainX_nominal, axis=0))
clusters_normal = kmeans_normal.predict(np.average(trainX_nominal,axis=0))

kmeans_enc = KMeans(n_clusters=4, random_state=0).fit(tsne_enc_df)
clusters_enc = kmeans_enc.predict(tsne_enc_df)

kmeans_dec = KMeans(n_clusters=4, random_state=0).fit(tsne_dec_df)
clusters_dec = kmeans_dec.predict(tsne_dec_df)

In [319]:
def transform_to_mask(cl_label, clust_obj):
    mask = []
    for i, elem in enumerate(clust_obj):
        if elem == cl_label:
            mask.append(i)
        else:
            pass
    return mask


def get_mask_list(clust_obj):
    mask_list = []
    for i in cluster_comp:
        mask_list.append(transform_to_mask(i, clust_obj))
    return mask_list


masks_normal = np.array(get_mask_list(clusters_normal))

masks_enc = np.array(get_mask_list(clusters_enc))

masks_dec = np.array(get_mask_list(clusters_dec))

masks = (masks_enc,masks_dec)
print(masks_normal.shape, masks_enc.shape, masks_enc.shape)


(4,) (4,) (4,)


In [320]:
# dist_list = []
# def calc_distance(elem, last_elem):
#     dist = abs(elem[0]-last_elem[0]) + abs(elem[1]-last_elem[1])
#     dist_list.append(dist)
#     return dist
# 
# 
# def plot_cl(cl, color):
#     parts = []
#     part = []
#     for i,elem in enumerate(cl):
#         if i == 0:
#             part.append(elem)  
#         elif calc_distance(elem, part[-1]) < 2:
#             part.append(elem)
#         else:
#             parts.append(part)
#             part = [elem]
#             part.append(elem)
#     for i in parts:
#         i = pd.DataFrame(i, columns=boat_csv.columns)
#         #print(i)
#         plt.plot(i["Lon"], i["Lat"], color=color)
def plot_cl(cl, color):
    plt.plot(cl["Lon"], cl["Lat"], color=color)
        
    
def plot_clusters_on_map():
    map = normal_data
    print(map.shape)
    for mask in masks:
        cl1 = pd.DataFrame(map[mask[0]], columns=boat_csv.columns)
        cl2 = pd.DataFrame(map[mask[1]], columns=boat_csv.columns)
        cl3 = pd.DataFrame(map[mask[2]], columns=boat_csv.columns)
        cl4 = pd.DataFrame(map[mask[3]], columns=boat_csv.columns)
       
        plot_cl(cl1, 'black')
        plot_cl(cl2, 'red')
        plot_cl(cl3, 'blue')
        plot_cl(cl4, 'green')
           
        plt.show()
        
        
plot_clusters_on_map()
plt.show()

(66963, 7)
